<a href="https://colab.research.google.com/github/RaghadQadah/Automatic-Spam-Review-Detection/blob/main/featureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt
import seaborn as sns
import re
#nltk.download('stopwords') for downloading first time uncomment this one
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from collections import Counter
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def word_count(df, text, name):
    df[name] = df[text].apply(lambda x: len(str(x).split(" ")))
    return df

def hashtags_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('#')]))
    return df

def user_mentions_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('@')]))
    return df

def numerics_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.isdigit()]))
    return df

def links_count(df, text, name):
    df[name] = df[text].apply(
        lambda x: len([x for x in  str(x).split() if x.startswith(('httP', 'HTTPS', 'https', 'www', 'WWW', 'HTTP', 'http')) ]))
    return df

def clean(df, text, name):
    df[name] = df[text].str.lower()
    df[name] = df[name].str.replace('http\S+|www.\S+', '', case=False)
    df[name] = df[name].str.replace('[^\w\s]', '')
    df[name] = df[name].str.replace('\d+', '')
    df[name] = df[name].apply(lambda x: ' '.join([word for word in  str(x).split() if word not in (stop)]))
    df[name] = df[name].apply(lambda x: " ".join(x for x in  str(x).split() if len(x) > 3))
    return df


In [ ]:
df = pd.read_csv("drive/My Drive/Output.csv",encoding='latin1',low_memory=False)
list_of_rows = [list(row) for row in df.values]

links_count(df,"reviewContent","linksCount")
hashtags_count(df, "reviewContent", "hashtagsCount")
user_mentions_count(df, "reviewContent", "userMentionsCount")
numerics_count(df,"reviewContent","numericsCount")


clean(df, "reviewContent", "cleanContent")
word_count(df, "cleanContent", "wordCount")




,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,Filtered,linksCount,hashtagsCount,userMentionsCount,numericsCount,cleanContent,wordCount
0,---2eJpQ2HAnRQRp2v7XLA,-ORdirlxcallT7dzlZlW2A,I came to Chicago on business and was initiall...,5,5,9/28/2011,-ORdirlxcallT7dzlZlW2A,0.0,2.0,0.0,0,0,0,0,came chicago business initially supposed stay ...,70
1,--0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268.0,543.0,0.0,0,0,0,0,still love place took menu deeply saddened sho...,10
2,--1PwTt0AfyPG1pE7bB6Nw,pW9LvtC-hZ0PRAOUn7aZ2A,Alan is the best CPA that I know. He knows all...,5,0,8/15/2006,pW9LvtC-hZ0PRAOUn7aZ2A,834.0,1463.0,0.0,0,0,0,0,alan best know knows outs loop holes smart whi...,28
3,--87QVdPvgbDK3GeVxFRzg,92rtlWxS2aOngzkKhi-OxQ,My family has been using Korean Air for many m...,4,0,7/28/2012,92rtlWxS2aOngzkKhi-OxQ,43.0,158.0,0.0,0,0,0,0,family using korean many many years going asia...,62
4,--99ZJoVHP32iSd4i4nlZw,Egu4qSFR56EhGuTHZTaVjA,I hadn't been to Grisanti's since a pre-homeco...,3,0,1/28/2010,Egu4qSFR56EhGuTHZTaVjA,133.0,725.0,0.0,0,0,0,0,hadnt grisantis since prehomecoming dinner hig...,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115472,__xtQo5KPZwjoK5JVN7Tqg,Y4Du3cllZ_gpNjzTvmeZ0Q,Five Stars cause they are the makers of Crysta...,5,2,6/30/2010,Y4Du3cllZ_gpNjzTvmeZ0Q,206.0,562.0,0.0,0,0,0,0,five stars cause makers crystal sauce single g...,44
115473,__yfM8bFN54IlsQ7EbF_vg,zHvS1F_6wOPSLUk6vpDEGA,I have stopped by here several times when hung...,4,3,11/6/2011,zHvS1F_6wOPSLUk6vpDEGA,35.0,502.0,0.0,0,0,0,0,stopped several times hunger hits wicker park ...,43
115474,__YLqReNU93Tg5b4-Iwbow,9dn3j_YcL8wl5jkwsFvyMw,Glad to see the positive reviews. Â If you th...,5,1,8/29/2012,9dn3j_YcL8wl5jkwsFvyMw,0.0,6.0,0.0,0,0,0,0,glad positive reviews think phase coolwait til...,19
115475,__YyIjBlNi33ZN-QynruNw,SKlqDjD5qJt9Kkm4KZCptw,If you're in the mood for a nice greasy pastra...,3,1,9/17/2012,SKlqDjD5qJt9Kkm4KZCptw,12.0,122.0,0.0,0,0,0,0,youre mood nice greasy pastrami sandwich find ...,27


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# X = vectorizer.fit_transform(df["cleanContent"]).toarray()
# print(X[0])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(df["cleanContent"]).toarray()
print(X)

[[0.11665971 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.17892585 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
dateList=[]
for word in list_of_rows:
    if "Update" in str(word[5]):
        dateList.append(1)
    else:
        dateList.append(0)

extremeRating=[]
for rating in list(df["rating"]):
    if rating == 5 or rating == 1:
        extremeRating.append(1)
    else:
        extremeRating.append(0)

reviewLengthSpam=[]
for c in df['reviewCount']:
    if c>150:
        reviewLengthSpam.append(0)
    else:
        reviewLengthSpam.append(1)



dict = {'UpdateOrNot': dateList,
       'NumberOfURLs': list(df["linksCount"]),
       'NumberOfnumerics': list(df["numericsCount"]),
       'extremeRating':extremeRating,
       'firstCount':list(df["firstCount"]),
       'reviewLengthSpam':reviewLengthSpam,
       'usefulCount':list(df['usefulCount']),
       'Filtered':list(df["Filtered"])
        }


df1 = pd.DataFrame(dict)
df1.to_csv('Data.csv',index=False)
df1.head()

,UpdateOrNot,NumberOfURLs,NumberOfnumerics,extremeRating,firstCount,reviewLengthSpam,usefulCount,Filtered
0,0,0,0,0,0.0,1,5,0.0
1,1,0,0,0,268.0,0,2,0.0
2,0,0,0,0,834.0,0,0,0.0
3,0,0,0,0,43.0,0,0,0.0
4,0,0,0,0,133.0,0,0,0.0
